# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Consumer** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://80d04dce9402:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c6cdd79-9dc1-4f44-8d55-d68651dfdc69;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "8e35f34f36db:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del stream

In [5]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(60)


25/05/01 05:17:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-115f409e-aa99-43a5-a2a3-35979a087549. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/01 05:17:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/01 05:17:18 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



25/05/01 05:17:21 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3255 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|key |value                                                     |topic              |partition|offset|timestamp              |timestampType|value_str          |
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|NULL|[7B 22 70 72 69 63 65 22 3A 20 39 34 39 36 30 2E 30 31 7D]|kafka-spark-example|0        |3998  |2025-05-01 05:17:24.974|0            |{"price": 94960.01}|
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+



25/05/01 05:17:31 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 4205 milliseconds


...

-------------------------------------------
Batch: 10
-------------------------------------------
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+
|key |value                                            |topic              |partition|offset|timestamp              |timestampType|value_str       |
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+
|NULL|[7B 22 70 72 69 63 65 22 3A 20 39 34 39 36 30 7D]|kafka-spark-example|0        |4007  |2025-05-01 05:18:10.021|0            |{"price": 94960}|
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+

-------------------------------------------
Batch: 11
-------------------------------------------
+----+-------------------------------------------------+--

-------------------------------------------
Batch: 14
-------------------------------------------
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|key |value                                                     |topic              |partition|offset|timestamp              |timestampType|value_str          |
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|NULL|[7B 22 70 72 69 63 65 22 3A 20 39 34 39 35 34 2E 35 35 7D]|kafka-spark-example|0        |4011  |2025-05-01 05:18:30.044|0            |{"price": 94954.55}|
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+

-------------------------------------------
Batch: 15
------------------------------------------

-------------------------------------------
Batch: 17
-------------------------------------------
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|key |value                                                     |topic              |partition|offset|timestamp              |timestampType|value_str          |
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+
|NULL|[7B 22 70 72 69 63 65 22 3A 20 39 34 39 35 31 2E 36 39 7D]|kafka-spark-example|0        |4014  |2025-05-01 05:18:45.062|0            |{"price": 94951.69}|
+----+----------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
query.stop()

25/04/10 02:04:09 ERROR TaskSchedulerImpl: Lost executor 1 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 02:04:09 ERROR TaskSchedulerImpl: Lost executor 0 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:09:30 ERROR TaskSchedulerImpl: Lost executor 3 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:09:30 ERROR TaskSchedulerImpl: Lost executor 2 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:46:21 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 2184299 milliseconds
25/04/10 03:57:42 ERROR TaskSchedulerImpl: Lost executor 5 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:57:42 ERROR TaskSchedulerImpl: Lost executor 4 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 04:07:38 ERROR TaskSchedulerImpl: Lost executor 7 on 172.24.0.5: worker lost: Not

In [6]:
sc.stop()